# Symbolic Gradient Calculation for SABR and SVI 
#### Dmitry Bazanov, Roland Grinis

In [3]:
import sympy as sym

## 1. SABR 

Implied volatility is given by:

$$ \sigma_B(x, T) \approx I_B^0(x) (1 + I_H^1(x)T), \quad x = \ln \frac{f}{K}$$

$$I_H^1(x) = \frac{\alpha^{2} \left(K f\right)^{\beta - 1} \left(1 - \beta\right)^{2}}{24} + \frac{\alpha \beta \rho v \left(K f\right)^{\frac{\beta}{2} - \frac{1}{2}}}{4} + \frac{v^{2} \cdot \left(2 - 3 \rho^{2}\right)}{24}$$

The order zero term is treated below.


In [4]:
alpha, beta, v, rho, f, K, T = sym.symbols('alpha beta v rho f K T')
x = sym.log(f/K)
I_H_1 = (((1 - beta)**2)*alpha**2/(24*(f*K)**(1 - beta)) + (rho*v*alpha*beta)/(4*(f*K)**((1-beta)/2)) + 
 ((2 - 3*rho**2)*v**2)/24 )

Derivatives of $I_H^1(x)$:

In [6]:
I_H_1.diff(alpha) # vega
I_H_1.diff(rho) # rega
I_H_1.diff(v) # sega
I_H_1.diff(f) # delta
I_H_1.diff(f).diff(f) # gamma
I_H_1.diff(f).diff(alpha) # gamma, vanna, volga
I_H_1.diff(alpha).diff(alpha) # volga
I_H_1.diff(K); # map to delta space

Derivatives of $I_H^0(x)$:

$\textbf{Case 1}: x = 0$ 

In [7]:
I_B_x = alpha*K**(beta-1)
I_B_x

K**(beta - 1)*alpha

In [8]:
I_B_x.diff(alpha) # vega
I_B_x.diff(rho) # rega
I_B_x.diff(v) # sega
I_B_x.diff(f) # delta
I_B_x.diff(f).diff(f) # gamma
I_B_x.diff(f).diff(alpha) # gamma, vanna, volga
I_B_x.diff(alpha).diff(alpha) # volga
sym.simplify(I_B_x.diff(K)); # map to delta space

$\textbf{Case 2}: v = 0$ 

In [9]:
I_B_v = x*alpha*(1 - beta)/(f**(1 - beta) - K**(1 - beta))
I_B_v

alpha*(1 - beta)*log(f/K)/(-K**(1 - beta) + f**(1 - beta))

In [10]:
I_B_v.diff(alpha).subs(x, "x") # vega
I_B_v.diff(rho) # rega
I_B_v.diff(v) # sega
sym.simplify(I_B_v.diff(f).subs(x, "x")) # delta
sym.simplify(I_B_v.diff(f).diff(f).subs(x, "x")) # gamma
I_B_v.diff(f).diff(alpha).subs(x, "x") # gamma, vanna, volga
I_B_v.diff(alpha).diff(alpha) # volga
sym.simplify(I_B_v.diff(K).subs(x, "x")); # map to delta space

$\textbf{Case 3}: \beta = 1$

In [11]:
z_b = v*x/alpha
xi_b = sym.sqrt(- 2*rho*z_b+z_b**2+1 )
I_B_b = v*x/sym.ln((xi_b + z_b - rho)/(1 - rho))
I_B_b

v*log(f/K)/log((-rho + sqrt(1 - 2*rho*v*log(f/K)/alpha + v**2*log(f/K)**2/alpha**2) + v*log(f/K)/alpha)/(1 - rho))

In [12]:
# vega
sym.simplify(I_B_b.diff(alpha).subs(xi_b, sym.Symbol(r"\xi")).subs(z_b, "z").subs(x, "x"))
# rega
sym.simplify(I_B_b.diff(rho).subs(xi_b, sym.Symbol(r"\xi")).subs(z_b, "z").subs(x, "x"))
# sega
sym.simplify(I_B_b.diff(v).subs(xi_b, sym.Symbol(r"\xi")).subs(z_b, "z").subs(x, "x"))
# delta
sym.simplify(I_B_b.diff(f).subs(xi_b, sym.Symbol(r"\xi")).subs(z_b, "z").subs(x, "x"))
# gamma
sym.simplify(I_B_b.diff(f).diff(f).subs(xi_b, sym.Symbol(r"\xi")).subs(z_b, "z").subs(x, "x"))
# gamma, vanna, volga
sym.simplify(I_B_b.diff(f).diff(alpha).subs(xi_b, sym.Symbol(r"\xi")).subs(z_b, "z").subs(x, "x"))
# volga
sym.simplify(I_B_b.diff(alpha).diff(alpha).subs(xi_b, sym.Symbol(r"\xi")).subs(z_b, "z").subs(x, "x"))
# map to delta space
sym.simplify(I_B_b.diff(K).subs(xi_b, sym.Symbol(r"\xi")).subs(z_b, "z").subs(x, "x"));

$\textbf{Case 4}: \beta < 1$

In [13]:
z = v*(f**(1 - beta) - K**(1 - beta))/(alpha * (1 - beta))
xi = sym.sqrt(- 2*rho*z+z**2+1 )
I_B_0 = v*x/sym.ln((xi + z - rho)/(1 - rho))
I_B_0

v*log(f/K)/log((-rho + sqrt(1 - 2*rho*v*(-K**(1 - beta) + f**(1 - beta))/(alpha*(1 - beta)) + v**2*(-K**(1 - beta) + f**(1 - beta))**2/(alpha**2*(1 - beta)**2)) + v*(-K**(1 - beta) + f**(1 - beta))/(alpha*(1 - beta)))/(1 - rho))

In [14]:
# vega
sym.simplify(I_B_0.diff(alpha).subs(xi, sym.Symbol(r"\xi")).subs(z, "z").subs(x, "x"))
# rega
sym.simplify(I_B_0.diff(rho).subs(xi, sym.Symbol(r"\xi")).subs(z, "z").subs(x, "x"))
# sega
sym.simplify(I_B_0.diff(v).subs(xi, sym.Symbol(r"\xi")).subs(z, "z").subs(x, "x"))
# delta
sym.simplify(I_B_0.diff(f).subs(xi, sym.Symbol(r"\xi")).subs(z, "z").subs(x, "x"))
# gamma
sym.simplify(I_B_0.diff(f).diff(f).subs(xi, sym.Symbol(r"\xi")).subs(z, "z").subs(x, "x"))
# gamma, vanna, volga
sym.simplify(I_B_0.diff(f).diff(alpha).subs(xi, sym.Symbol(r"\xi")).subs(z, "z").subs(x, "x"))
# volga
sym.simplify(I_B_0.diff(alpha).diff(alpha).subs(xi, sym.Symbol(r"\xi")).subs(z, "z").subs(x, "x"))
# map to delta space
sym.simplify(I_B_0.diff(K).subs(xi, sym.Symbol(r"\xi")).subs(z, "z").subs(x, "x"));

Implied volatility senstivities are easy to obtain:
$$ \nabla_{\alpha, \rho, v, f, K} \sigma_B(x, T) =  \nabla_{\alpha, \rho, v, f, K} I_B^0(x) \cdot \left(1 + I_H^1(x)T \right) + I_B^0(x) \cdot T \cdot \nabla_{\alpha, \rho, v, f, K}I_H^1(x)
$$

$$
\nabla_{\alpha, f}^2 \sigma_B= \nabla_{\alpha, f}^2 I_B^0(x) + T\left( \nabla_{\alpha,f}^2 I_B^0(x) \cdot I_H^1(x) +  \nabla_{\alpha, f}^2  I_H^1(x) \cdot I_B^0(x)  + 2 \nabla_{\alpha, f} I_B^0(x) \cdot \nabla_{\alpha, f} I_H^1(x) \right)
$$

$$ \frac{\partial ^2 \sigma_B}{\partial \alpha \partial f} = \frac{\partial^2 I_B^0(x)}{\partial \alpha \partial f} (1 + I_H^1(x)T) + \frac{\partial I_B^0(x)}{\partial \alpha} \frac{\partial I_H^1(x)}{\partial f}T + \frac{\partial^2 I_H^1(x)}{\partial \alpha \partial f} I_B^0(x) T +\frac{\partial I_H^1(x)}{\partial \alpha} \frac{\partial I_B^0(x)}{\partial f}T $$